In [3]:
import requests 
from bs4 import BeautifulSoup
import os
import glob
import pandas as pd
import hashlib
import random
import json

import OAI

import numpy as np

from lxml import html
from bs4 import BeautifulSoup
import trafilatura
import io

import os, openai, datetime, hashlib, re
import time
from langchain.document_loaders import DataFrameLoader
from langchain.vectorstores import Chroma

from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import Chroma
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

import pandas as pd

import string
LETTERS = string.ascii_uppercase

In [1]:
import helpers as s

BASE_URL = "substack.kghosh.me"
pages = s.get_cache(BASE_URL)


102


In [2]:
s.do_meta()


Error with: .cache_keywords/8cab491fb7be16a880b6d24655941e75
Error with: .cache_keywords/2b4dbdd486d600d190e593ea00462591
Error with: .cache_keywords/7c4551fb2f96cce7c0264d6b1f50b669
Error with: .cache_keywords/c21f7a31bf1c99ef78e154ae4249e712
Error with: .cache_keywords/fc20f6ec9a33190995ea38eb55535af3
Error with: .cache_keywords/69203e4b629da703a42daa81e7f87ab9
Error with: .cache_keywords/d91570978a514e8aedb83c47a45c169d
Error with: .cache_keywords/52ee8c1b0291daa92b97c28225a816b6
Error with: .cache_keywords/90902a39015be5640604e94bc14b555d
Error with: .cache_keywords/58f70468586e816cc19aaf2bfbbf8f5b
Error with: .cache_keywords/f157f8388cf71104713811b59d220bc0
Error with: .cache_keywords/42624b772724c675d044f1ca4e3ab624


AttributeError: 'float' object has no attribute 'split'

In [4]:
from helpers import txtRead

In [5]:
SEEDS = list(glob.glob(".cache_keywords/*"))
srcFiles = pd.read_parquet('data/urls.parquet.gzip')
srcFiles.columns = ["origin","url","src"]
allSeeds = []
for seed in SEEDS:
    try:
        txt = txtRead(seed)
        txt.replace("\n\n","\n")
        SUMMARY = txt.split("\nSummary:")[-1].replace("\n"," ").strip()
        C = txt.split("\n")
        TH = [x[8:].replace("'","").replace("[","").replace("]","") for x in C if "THEMES = " in x][0].strip().split(",")
        TH = [x.strip() for x in TH]
        KW = [x[10:].replace("'","").replace("[","").replace("]","") for x in C if "KEYWORDS = " in x][0].strip().split(",")
        KW = [x.strip() for x in KW]
        ID = seed.split("/")[-1]
        allSeeds.append([ID,KW,TH,SUMMARY])
    except:
        print("Error with:",seed)
META = pd.DataFrame(allSeeds)
META.columns = ["src","keywords","themes","summary"]
META = META.merge(srcFiles,on="src",how="left")
META

Error with: .cache_keywords/8cab491fb7be16a880b6d24655941e75
Error with: .cache_keywords/2b4dbdd486d600d190e593ea00462591
Error with: .cache_keywords/7c4551fb2f96cce7c0264d6b1f50b669
Error with: .cache_keywords/c21f7a31bf1c99ef78e154ae4249e712
Error with: .cache_keywords/fc20f6ec9a33190995ea38eb55535af3
Error with: .cache_keywords/69203e4b629da703a42daa81e7f87ab9
Error with: .cache_keywords/d91570978a514e8aedb83c47a45c169d
Error with: .cache_keywords/52ee8c1b0291daa92b97c28225a816b6
Error with: .cache_keywords/90902a39015be5640604e94bc14b555d
Error with: .cache_keywords/58f70468586e816cc19aaf2bfbbf8f5b
Error with: .cache_keywords/f157f8388cf71104713811b59d220bc0
Error with: .cache_keywords/42624b772724c675d044f1ca4e3ab624


,src,keywords,themes,summary,origin,url
0,c94f72ff677c7517a836417c1f1df620,"[AI, work, consultants, tasks, performance, Ja...","[Impact of AI on work, The Jagged Frontier of ...","In this post, the author discusses the impact ...",.cache/20230927,b'https://www.oneusefulthing.org/p/centaurs-an...
1,e88f2221b01e6b71b610a1e6c9e2c576,"[Snailbrook, Texas, Elon Musk, Tesla, Austin, ...","[Company towns, Urban development, Tech billio...","Snailbrook, Texas, is a town that is being dev...",.cache/20230612,b'https://www.theguardian.com/society/2023/apr...
2,656239ad8ab36ce06ac4ee6258b5381d,"[Atlas for The Blind, 1837, blind, embossed, p...","[education, disabilities, geography]","The Atlas for the Blind, published in 1837, wa...",.cache/20230616,b'https://www.davidrumsey.com/blog/2012/5/21/a...
3,93cdfad04e16027123b398b4675b51da,"[eagles, migration route, Ukraine war, habitat...","[wildlife migration, impact of war on wildlife...",Eagles have changed their migration routes acr...,.cache/20240609,b'https://www.bbc.com/news/articles/c88z0p33413o'
4,c7d3021c1b6298e7c47bc455cd7d8391,"[Nature-based Solutions, Scotland, climate cha...","[Nature-based Solutions, Climate Change, Urban...",This text discusses the importance of Nature-b...,.cache/20040714,b'https://darkmatterlabs.org/Urban-Nature-base...
...,...,...,...,...,...,...
2132,a6bfcabefea560d7dfefd5aab87f047c,"[physicist, nanoscience lab, cooling paint, vi...","[nanoscience, structural color, environmentall...",Debashis Chanda and his team at the University...,.cache/20230408,b'https://www.wired.com/story/lightest-paint-i...
2133,5d04744a3bee2ab6564f5e2ff60136d8,"[knowledge graphs, unstructured text, LLMs, da...","[Knowledge extraction, Unstructured data proce...",This blog post discusses the use of Large Lang...,.cache/20231017,b'https://medium.com/neo4j/construct-knowledge...
2134,037aba804501ec9f75f8bb434b8a6c20,"[learn in public, learning, content, create, b...","[learning in public, creating content, persona...",Learning in Public emphasizes the importance o...,.cache/20220717,b'https://www.swyx.io/learn-in-public/'
2135,382e9ebc1e518ee49e541da1e6b5f8af,"[OpenAI, large language model, ChatGPT, global...","[AI research, AI ethics and governance, regula...",OpenAI's introduction of its large language mo...,.cache/20230604,b'https://spectrum.ieee.org/ai-ethics-governance'


In [9]:
META[META.origin.isna()]

,src,keywords,themes,summary,origin,url
629,04be88385ee32faf2742b50c6f1b32f7,"[Better Sleep, vintage sign, blanket support, ...","[vintage advertising, sleep products, collecti...",The text discusses a vintage advertisement for...,NaN,NaN
